# Experiment Records

## expriment id = 0: 
```
        self.dimension_reduce_layer = nn.Linear(32, 64)
        
        self.mlp = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 2)  # 2 classes: real disaster or not
        )
```
train:
```
train epoch 0/20
100%|██████████| 167/167 [01:47<00:00,  1.55it/s]
100%|██████████| 167/167 [00:42<00:00,  3.89it/s]
Validation F1 Score: 83.95%
train epoch 1/20
100%|██████████| 167/167 [01:42<00:00,  1.62it/s]
train epoch 2/20
100%|██████████| 167/167 [01:47<00:00,  1.56it/s]
100%|██████████| 167/167 [00:45<00:00,  3.65it/s]
Validation F1 Score: 89.31%
train epoch 3/20
100%|██████████| 167/167 [01:40<00:00,  1.66it/s]
train epoch 4/20
100%|██████████| 167/167 [01:37<00:00,  1.71it/s]
100%|██████████| 167/167 [00:53<00:00,  3.15it/s]
Validation F1 Score: 92.16%
train epoch 5/20
100%|██████████| 167/167 [01:35<00:00,  1.74it/s]
train epoch 6/20
100%|██████████| 167/167 [01:31<00:00,  1.83it/s]
100%|██████████| 167/167 [00:43<00:00,  3.81it/s]
Validation F1 Score: 93.37%
train epoch 7/20
100%|██████████| 167/167 [01:39<00:00,  1.68it/s]
train epoch 8/20
100%|██████████| 167/167 [01:32<00:00,  1.81it/s]
100%|██████████| 167/167 [00:44<00:00,  3.77it/s]
Validation F1 Score: 94.57%
train epoch 9/20
100%|██████████| 167/167 [01:32<00:00,  1.81it/s]
train epoch 10/20
100%|██████████| 167/167 [01:43<00:00,  1.61it/s]
100%|██████████| 167/167 [00:42<00:00,  3.90it/s]
Validation F1 Score: 94.31%
train epoch 11/20
100%|██████████| 167/167 [01:34<00:00,  1.78it/s]
train epoch 12/20
100%|██████████| 167/167 [01:31<00:00,  1.82it/s]
100%|██████████| 167/167 [00:43<00:00,  3.83it/s]
Validation F1 Score: 97.63%
train epoch 13/20
100%|██████████| 167/167 [01:37<00:00,  1.72it/s]
train epoch 14/20
100%|██████████| 167/167 [01:29<00:00,  1.87it/s]
100%|██████████| 167/167 [00:42<00:00,  3.97it/s]
Validation F1 Score: 97.49%
train epoch 15/20
100%|██████████| 167/167 [01:30<00:00,  1.85it/s]
train epoch 16/20
100%|██████████| 167/167 [01:28<00:00,  1.90it/s]
100%|██████████| 167/167 [00:39<00:00,  4.27it/s]
Validation F1 Score: 98.09%
train epoch 17/20
100%|██████████| 167/167 [01:32<00:00,  1.80it/s]
train epoch 18/20
100%|██████████| 167/167 [01:31<00:00,  1.83it/s]
100%|██████████| 167/167 [00:40<00:00,  4.09it/s]
Validation F1 Score: 97.78%
train epoch 19/20
100%|██████████| 167/167 [01:35<00:00,  1.74it/s]
```
test:
```
0.8756477673026409
```

## Experiment ID = 1

```
class DisasterClassifier(nn.Module):
    def __init__(self):
        super(DisasterClassifier, self).__init__()
        
        self.bert = AutoAdapterModel.from_pretrained('bert-base-uncased')
        for param in self.bert.parameters():
            param.requires_grad = False
        # Load pre-trained task adapter from Adapter Hub
        # This method call will also load a pre-trained classification head for the adapter task  
        
        # https://github.com/adapter-hub/adapter-transformers/blob/master/adapter_docs/prediction_heads.md
        self.bert_out_dim = 32
        self.bert.add_classification_head("mrpc", num_labels=self.bert_out_dim)
        
        self.bert.add_adapter("mrpc", config="pfeiffer")
        self.bert.set_active_adapters("mrpc")
        
        # check the reduction factor for "sentiment/sst-2@ukp": https://adapterhub.ml/adapters/ukp/bert-base-uncased_sentiment_sst-2_pfeiffer/
        # bert output dimension = 786/reduction factor
        self.adpter_reduction_factor = 16
        # self.dimension_reduce_layer = nn.Linear(int(768/self.adpter_reduction_factor), 64)  # Reduce BERT's output dimension
        self.dimension_reduce_layer = nn.Linear(self.bert_out_dim, 64)
        
        self.mlp = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 2)  # 2 classes: real disaster or not
        )

    def forward(self, input_ids):
        outputs = self.bert(input_ids)
        pooled_output = outputs['logits']
        pooled_output = self.dimension_reduce_layer(pooled_output)
        logits = self.mlp(pooled_output)
        return logits
```
trian, validate and test:
```
train epoch 0/20
100%|██████████| 167/167 [02:22<00:00,  1.17it/s]
100%|██████████| 119/119 [00:49<00:00,  2.40it/s]
Validation F1 Score: 82.43%
100%|██████████| 119/119 [00:40<00:00,  2.95it/s]
Test F1 Score: 82.45%
train epoch 1/20
100%|██████████| 167/167 [02:23<00:00,  1.16it/s]
train epoch 2/20
100%|██████████| 167/167 [02:22<00:00,  1.17it/s]
100%|██████████| 119/119 [00:51<00:00,  2.30it/s]
Validation F1 Score: 80.12%
100%|██████████| 119/119 [00:47<00:00,  2.49it/s]
Test F1 Score: 78.59%
train epoch 3/20
100%|██████████| 167/167 [02:23<00:00,  1.16it/s]
train epoch 4/20
100%|██████████| 167/167 [02:23<00:00,  1.16it/s]
100%|██████████| 119/119 [00:50<00:00,  2.36it/s]
Validation F1 Score: 87.93%
100%|██████████| 119/119 [00:47<00:00,  2.52it/s]
Test F1 Score: 84.16%
train epoch 5/20
100%|██████████| 167/167 [02:24<00:00,  1.15it/s]
train epoch 6/20
100%|██████████| 167/167 [02:23<00:00,  1.17it/s]
100%|██████████| 119/119 [00:48<00:00,  2.46it/s]
Validation F1 Score: 91.51%
100%|██████████| 119/119 [00:39<00:00,  3.01it/s]
Test F1 Score: 84.99%
train epoch 7/20
100%|██████████| 167/167 [02:21<00:00,  1.18it/s]
train epoch 8/20
100%|██████████| 167/167 [02:24<00:00,  1.16it/s]
100%|██████████| 119/119 [00:46<00:00,  2.55it/s]
Validation F1 Score: 89.48%
100%|██████████| 119/119 [00:45<00:00,  2.60it/s]
Test F1 Score: 83.81%
train epoch 9/20
100%|██████████| 167/167 [02:23<00:00,  1.16it/s]
train epoch 10/20
100%|██████████| 167/167 [02:26<00:00,  1.14it/s]
100%|██████████| 119/119 [00:49<00:00,  2.40it/s]
Validation F1 Score: 94.37%
100%|██████████| 119/119 [00:50<00:00,  2.36it/s]
Test F1 Score: 84.98%
train epoch 11/20
100%|██████████| 167/167 [02:24<00:00,  1.16it/s]
train epoch 12/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
100%|██████████| 119/119 [00:51<00:00,  2.33it/s]
Validation F1 Score: 94.88%
100%|██████████| 119/119 [00:47<00:00,  2.52it/s]
Test F1 Score: 84.48%
train epoch 13/20
100%|██████████| 167/167 [02:26<00:00,  1.14it/s]
train epoch 14/20
100%|██████████| 167/167 [02:22<00:00,  1.17it/s]
100%|██████████| 119/119 [00:47<00:00,  2.49it/s]
Validation F1 Score: 96.91%
100%|██████████| 119/119 [00:43<00:00,  2.76it/s]
Test F1 Score: 86.71%
train epoch 15/20
100%|██████████| 167/167 [02:21<00:00,  1.18it/s]
train epoch 16/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
100%|██████████| 119/119 [00:47<00:00,  2.51it/s]
Validation F1 Score: 95.94%
100%|██████████| 119/119 [00:40<00:00,  2.95it/s]
Test F1 Score: 86.28%
train epoch 17/20
100%|██████████| 167/167 [02:19<00:00,  1.20it/s]
train epoch 18/20
100%|██████████| 167/167 [02:24<00:00,  1.16it/s]
100%|██████████| 119/119 [00:49<00:00,  2.41it/s]
Validation F1 Score: 97.53%
 68%|██████▊   | 81/119 [00:33<00:16,  2.33it/s]
 ```


# id=2
```
        # https://github.com/adapter-hub/adapter-transformers/blob/master/adapter_docs/prediction_heads.md
        self.bert_out_dim = 32
        self.bert.add_classification_head("mrpc", num_labels=self.bert_out_dim)
        
        self.bert.add_adapter("mrpc", config="pfeiffer")
        self.bert.set_active_adapters("mrpc")
        
        # self.dimension_reduce_layer = nn.Linear(int(768/self.adpter_reduction_factor), 64)  # Reduce BERT's output dimension
        self.dimension_reduce_layer = nn.Linear(self.bert_out_dim, 64)
        
        self.mlp = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 8),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(8, 2)  # 2 classes: real disaster or not
        )
```
train:
```
train epoch 0/20
100%|██████████| 167/167 [02:28<00:00,  1.12it/s]
100%|██████████| 119/119 [00:49<00:00,  2.38it/s]
Validation F1 Score: 83.11%
100%|██████████| 119/119 [00:34<00:00,  3.42it/s]
Test F1 Score: 82.42%
train epoch 1/20
100%|██████████| 167/167 [02:21<00:00,  1.18it/s]
train epoch 2/20
100%|██████████| 167/167 [02:23<00:00,  1.16it/s]
100%|██████████| 119/119 [00:47<00:00,  2.49it/s]
Validation F1 Score: 86.19%
100%|██████████| 119/119 [00:41<00:00,  2.86it/s]
Test F1 Score: 84.15%
train epoch 3/20
100%|██████████| 167/167 [02:18<00:00,  1.20it/s]
train epoch 4/20
100%|██████████| 167/167 [02:19<00:00,  1.19it/s]
100%|██████████| 119/119 [00:48<00:00,  2.44it/s]
Validation F1 Score: 86.48%
100%|██████████| 119/119 [00:50<00:00,  2.38it/s]
Test F1 Score: 83.03%
train epoch 5/20
100%|██████████| 167/167 [02:21<00:00,  1.18it/s]
train epoch 6/20
100%|██████████| 167/167 [02:27<00:00,  1.13it/s]
100%|██████████| 119/119 [00:51<00:00,  2.30it/s]
Validation F1 Score: 89.49%
100%|██████████| 119/119 [00:48<00:00,  2.44it/s]
Test F1 Score: 81.65%
train epoch 7/20
100%|██████████| 167/167 [02:28<00:00,  1.13it/s]
train epoch 8/20
100%|██████████| 167/167 [02:27<00:00,  1.13it/s]
100%|██████████| 119/119 [00:49<00:00,  2.39it/s]
Validation F1 Score: 93.51%
100%|██████████| 119/119 [00:49<00:00,  2.41it/s]
Test F1 Score: 86.53%
train epoch 9/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
train epoch 10/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
100%|██████████| 119/119 [00:49<00:00,  2.41it/s]
Validation F1 Score: 94.21%
100%|██████████| 119/119 [00:49<00:00,  2.42it/s]
Test F1 Score: 85.46%
train epoch 11/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
train epoch 12/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
100%|██████████| 119/119 [00:49<00:00,  2.42it/s]
Validation F1 Score: 96.47%
100%|██████████| 119/119 [00:49<00:00,  2.41it/s]
Test F1 Score: 87.55%
train epoch 13/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
train epoch 14/20
100%|██████████| 167/167 [02:26<00:00,  1.14it/s]
100%|██████████| 119/119 [00:50<00:00,  2.35it/s]
Validation F1 Score: 97.56%
100%|██████████| 119/119 [00:38<00:00,  3.06it/s]
Test F1 Score: 88.04%
train epoch 15/20
100%|██████████| 167/167 [02:18<00:00,  1.21it/s]
train epoch 16/20
100%|██████████| 167/167 [02:18<00:00,  1.20it/s]
100%|██████████| 119/119 [00:47<00:00,  2.49it/s]
Validation F1 Score: 96.74%
100%|██████████| 119/119 [00:48<00:00,  2.43it/s]
Test F1 Score: 85.73%
train epoch 17/20
100%|██████████| 167/167 [02:24<00:00,  1.16it/s]
train epoch 18/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
100%|██████████| 119/119 [00:49<00:00,  2.39it/s]
Validation F1 Score: 97.30%
100%|██████████| 119/119 [00:50<00:00,  2.35it/s]
Test F1 Score: 86.73%
train epoch 19/20
100%|██████████| 167/167 [02:25<00:00,  1.15it/s]
```